In [ ]:
import os
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from einops import rearrange
from gluonts.dataset.pandas import PandasDataset, is_uniform, infer_freq
from gluonts.dataset.split import split
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule
from uni2ts.eval_util.plot import plot_single, plot_next_multi

from fts_explore.common.benchmark_func import *

In [ ]:
SZ = "small"
PDT = 7 * 24  # prediction length: any positive integer
CTX = 1 * ((4 * PDT) + 48)  # context length: any positive integer
PSZ = 64  # patch size: choose from {"auto", 8, 16, 32, 64, 128}
BSZ = PDT  # batch size: any positive integer
TEST = 6 * ((4 * PDT) + 48)  # test set length: any positive integer

HISTORY = CTX
NUM_WINDOWS = TEST // PDT
DIST = PDT

In [ ]:
TEST

In [ ]:
moirai = MoiraiModule.from_pretrained("Salesforce/moirai-1.0-R-small")

In [ ]:
data = pd.read_csv("../data/btc_23_24.csv")
data = data.iloc[8760 - CTX : 8760 + TEST, :]
data = data.drop_duplicates("DateUTC")
data["DateUTC"] = pd.to_datetime(data["DateUTC"])
data = data.set_index("DateUTC")

In [ ]:
data.shape

In [ ]:
data.plot(figsize=(12, 3))
plt.tight_layout();

In [ ]:
# Prepare model
model = MoiraiForecast(
    module=moirai,
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=200,
    target_dim=1,
    feat_dynamic_real_dim=0,
    past_feat_dynamic_real_dim=0,
)

In [ ]:
ds = PandasDataset(data, target="BTC-USD", freq="H")

# Split into train/test set
train, test_template = split(
    ds, offset=-TEST
)  # assign last TEST time steps as test set

# Construct rolling window evaluation
test_data = test_template.generate_instances(
    prediction_length=PDT,  # number of time steps for each prediction
    windows=NUM_WINDOWS,  # number of windows in rolling window evaluation
    distance=DIST,  # number of time steps between each window - distance=PDT for non-overlapping windows
    max_history=HISTORY,
)

In [ ]:
forecast_samples, target_values = get_eval_foreasts(model, test_data)

In [ ]:
# mean_error = (
#     np.mean(forecast_samples, axis=1).flatten() - target_values.flatten()
# ) / target_values.flatten()
# upper_error = (
#     np.quantile(forecast_samples, 0.975, axis=1).flatten() - target_values.flatten()
# ) / target_values.flatten()
# lower_error = (
#     np.quantile(forecast_samples, 0.025, axis=1).flatten() - target_values.flatten()
# ) / target_values.flatten()

# window_size = 96
# mean_error_ma = moving_average(np.array(mean_error), window_size)
# upper_error_ma = moving_average(np.array(upper_error), window_size)
# lower_error_ma = moving_average(np.array(lower_error), window_size)

# plt.figure(figsize=(8, 8))
# plt.plot(
#     mean_error_ma,
#     label="mean",
# )

# plt.fill_between(
#     range(0, 25 * 168 - window_size + 1),
#     lower_error_ma,
#     upper_error_ma,
#     alpha=0.5,
#     label="a=0.05",
# )
# plt.legend();

In [ ]:
# plt.figure(figsize=(8, 8))
# plt.plot(
#     np.mean(forecast_samples, axis=1).flatten()[:168],
#     label="mean",
# )

# plt.fill_between(
#     range(0, 168),
#     np.quantile(forecast_samples, 0.025, axis=1).flatten()[:168],
#     np.quantile(forecast_samples, 0.975, axis=1).flatten()[:168],
#     alpha=0.5,
#     label="a=0.05",
# )
# plt.legend();

In [ ]:
predictor = model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data.input)

input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

In [ ]:
errors = []
for i in range(NUM_WINDOWS):
    try:
        label = next(label_it)["target"]
        forecast = next(forecast_it).mean
    except Exception as e:
        print(e)
        break

    errors.append(np.mean(np.abs((label - forecast) / label)))

In [ ]:
window_size = 4
plt.title(f"Pretrained MOIRAI MAPE: {np.round(np.mean(errors), 3)}")
plt.plot(
    range(window_size - 1, len(errors)),
    moving_average(np.array(errors), window_size),
    label="Error MA",
)
plt.plot(range(len(errors)), errors, label="Error")
plt.legend();
# plt.savefig("pretrained_moirai_mae.png");

In [ ]:
# # Make predictions
# if NUM_WINDOWS <= 30:
#     predictor = model.create_predictor(batch_size=BSZ)
#     forecasts = predictor.predict(test_data.input)

#     input_it = iter(test_data.input)
#     label_it = iter(test_data.label)
#     forecast_it = iter(forecasts)

#     fig, axes = plt.subplots(nrows=NUM_WINDOWS, ncols=1, figsize=(8, NUM_WINDOWS * 4))
#     plot_next_multi(
#         axes,
#         input_it,
#         label_it,
#         forecast_it,
#         context_length=1 * PDT,
#         intervals=(0.9, 0.95),
#         dim=None,
#         name="Prediction",
#         show_label=True,
#     )
#     plt.tight_layout()

#     plt.savefig("pretrained_moirai_predictions.png")

In [ ]:
experiment_run = "stage_one_crypto"
os.listdir(f"../outputs/finetune/{experiment_run}/checkpoints/")

In [ ]:
checkpoint = os.listdir(f"../outputs/finetune/{experiment_run}/checkpoints/")[-1]

print(f"Checkpoint: {checkpoint}")

fine_tuned_model = MoiraiForecast.load_from_checkpoint(
    prediction_length=PDT,
    context_length=CTX,
    patch_size=PSZ,
    num_samples=200,
    target_dim=1,
    feat_dynamic_real_dim=0,
    past_feat_dynamic_real_dim=0,
    checkpoint_path=f"../outputs/finetune/{experiment_run}/checkpoints/{checkpoint}",
)

In [ ]:
predictor = fine_tuned_model.create_predictor(batch_size=BSZ)
forecasts = predictor.predict(test_data.input)

input_it = iter(test_data.input)
label_it = iter(test_data.label)
forecast_it = iter(forecasts)

In [ ]:
finetuned_errors = []
for i in range(NUM_WINDOWS):
    try:
        label = next(label_it)["target"]
        forecast = next(forecast_it).mean
    except Exception as e:
        print(e)
        break

    finetuned_errors.append(np.mean(np.abs((label - forecast) / label)))

In [ ]:
window_size = 4
plt.title(f"Finetuned MOIRAI MAE: {np.round(np.mean(finetuned_errors), 3)}")
plt.plot(
    range(window_size - 1, len(errors)),
    moving_average(np.array(finetuned_errors), window_size),
    label="Error MA",
)
plt.plot(range(len(finetuned_errors)), finetuned_errors, label="Error")
plt.legend()
# plt.savefig("finetuned_moirai_mae.png");

In [ ]:
# # Make predictions
# if NUM_WINDOWS <= 30:
#     predictor = fine_tuned_model.create_predictor(batch_size=BSZ)
#     forecasts = predictor.predict(test_data.input)

#     input_it = iter(test_data.input)
#     label_it = iter(test_data.label)
#     forecast_it = iter(forecasts)

#     fig, axes = plt.subplots(nrows=NUM_WINDOWS, ncols=1, figsize=(8, NUM_WINDOWS * 4))
#     plot_next_multi(
#         axes,
#         input_it,
#         label_it,
#         forecast_it,
#         context_length=1 * PDT,
#         intervals=(0.9, 0.95),
#         dim=None,
#         name="Prediction",
#         show_label=True,
#     )
#     plt.tight_layout()
#     plt.savefig("finetuned_moirai_predictions.png")

In [ ]:
window_size = 4
plt.title("Finetuned vs Pretrained MOIRAI")
plt.plot(
    range(len(moving_average(np.array(errors), window_size))),
    moving_average(np.array(errors), window_size),
    label="Pretrained MA Error",
)
plt.plot(
    range(len(moving_average(np.array(finetuned_errors), window_size))),
    moving_average(np.array(finetuned_errors), window_size),
    label="Finetuned MA Error",
)
plt.xlabel("Training Window Index")
plt.ylabel("MAPE")
plt.legend()
# plt.savefig("../experiment_results/stage_1_finetune_gr_data/ft_vs_pt_train_mape.jpeg");

In [ ]:
# from uni2ts.eval_util.evaluation import evaluate_model
# from gluonts.ev.metrics import MAE, MAPE

In [ ]:
# metrics = []
# for perc in [0.025, 0.05, 0.1, 0.5, 0.9, 0.95, 0.975]:
#     metric_eval = evaluate_model(
#         model=fine_tuned_model.create_predictor(batch_size=BSZ),
#         test_data=test_data,
#         metrics=[MAE(perc), MAPE(perc)],
#     )

#     metrics.append(metric_eval)

# pd.concat(metrics, axis=1)

In [ ]:
# metrics = []
# for perc in [0.025, 0.05, 0.1, 0.5, 0.9, 0.95, 0.975]:
#     metric_eval = evaluate_model(
#         model=model.create_predictor(batch_size=BSZ),
#         test_data=test_data,
#         metrics=[MAE(perc), MAPE(perc)],
#     )

#     metrics.append(metric_eval)

# pd.concat(metrics, axis=1)